# Installing the Tensorflow Object Detection API

#### Clone the repository and install dependencies.

In [1]:
import tensorflow as tf

tf.__version__

'2.2.0'

In [2]:
!git clone -q https://github.com/tensorflow/models.git
!pip install -qU Cython contextlib2 pillow lxml jupyter matplotlib

     |████████████████████████████████| 2.2MB 4.0MB/s 
     |████████████████████████████████| 5.5MB 36.7MB/s 
     |████████████████████████████████| 11.5MB 41.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


#### Install the COCO API

In [3]:
!git clone -q https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../
%cp -r cocoapi/PythonAPI/pycocotools models/research/

/content/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I../common -I/usr/include/python3.6m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.6/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDe

#### Protobuf Compilation and Object Detection API installation

In [4]:
%%bash
cd models/research

protoc object_detection/protos/*.proto --python_out=.

cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1539742 sha256=2ec93fdc6eeccf47bf640b1999f32a12f1e95ad6548169a7e3df158cf1d39472
  Stored in directory: /tmp/pip-ephem-wheel-cache-_r6igvdx/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20069 sha256=188c56fbed18fcd57e156f058c92a2b5fb1e3915a3e7e0a5ed8482b6633c8d13
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built object-detection py-cpuinfo


object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


## Set paths

In [5]:
! export PYTHONPATH=$PYTHONPATH:/content/models/research
! export PYTHONPATH=$PYTHONPATH:/content/models/research/object_detection
! export PYTHONPATH=$PYTHONPATH:/content/models/research/slim

In [6]:
! python models/research/object_detection/builders/model_builder_tf2_test.py

2020-07-24 13:06:07.125746: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-07-24 13:06:10.299935: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-24 13:06:10.357635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-24 13:06:10.358458: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-24 13:06:10.358510: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfull

# Object detection

In [7]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [8]:
#@title Install libraries

!pip install -qU googledrivedownloader silence-tensorflow

## SETUP

In [9]:
#@title imports

import silence_tensorflow

import object_detection
import shutil
import os
import tarfile
import six.moves.urllib as urllib
import tensorflow as tf
ROOT_DIR = '/content'

os.chdir(ROOT_DIR)

In [21]:
# MODELS

# ---- EfficientDet D0
MODEL_TYPE = 'efficientdet_d0_coco17_tpu-32'        #---- 39	33.6
CONFIG_TYPE = 'ssd_efficientdet_d0_512x512_coco17_tpu-8'

# ---- EfficientDet D1
# MODEL_TYPE = 'efficientdet_d1_coco17_tpu-32'     # ----- 54	38.4
# CONFIG_TYPE = 'ssd_efficientdet_d1_640x640_coco17_tpu-8'

# ---- MOBILENET V2 FPNLITE 320                   #  ----- 22	22.2
# MODEL_TYPE = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
# CONFIG_TYPE = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'


NUM_TRAIN_STEPS =  40000

In [22]:
#@title Image_sizes
IMAGE_SIZE = {
    'efficientdet_d0_coco17_tpu-32': 512,
    'efficientdet_d1_coco17_tpu-32': 640,
    'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8': 320,
}

In [23]:
#@title path setup
IM_SZE = IMAGE_SIZE[MODEL_TYPE] 

WORKSPACE_PATH  = '/content/workspace/'
CHECKPOINT_DIR = f'/content/workspace/{MODEL_TYPE}'
OUTPUT_PATH     = '/content/workspace/output'
EXPORTED_PATH   = '/content/workspace/exported'
ANNOTATIONS_PATH = '/content/workspace/annotations'
IMAGES_PATH = f'/content/workspace/Images_{IM_SZE}'

LABEL_MAP_PATH      = os.path.join(WORKSPACE_PATH, 'label_map.pbtxt')
TRAIN_CSV_PATH      = os.path.join(ANNOTATIONS_PATH, f'training_{IM_SZE}.csv')
VALID_CSV_PATH      = os.path.join(ANNOTATIONS_PATH, f'validation_{IM_SZE}.csv')
TRAIN_RECORD_PATH   = os.path.join(ANNOTATIONS_PATH, f'train_{IM_SZE}.record')
VALID_RECORD_PATH   = os.path.join(ANNOTATIONS_PATH, f'val_{IM_SZE}.record')

In [24]:
#@title initiate directory 
if os.path.isdir(WORKSPACE_PATH):
    shutil.rmtree(WORKSPACE_PATH)    

os.makedirs(WORKSPACE_PATH)
os.makedirs(OUTPUT_PATH)
os.makedirs(EXPORTED_PATH)

In [25]:
#@title download data, configs, tfrecord generator - SMART INDIA HACKATHON

# https://drive.google.com/file/d/1L_CvmHYL4Z8mYmJZznNDGnB9G1hYMv3P/view?usp=sharing
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1L_CvmHYL4Z8mYmJZznNDGnB9G1hYMv3P',
                                    dest_path=f'{WORKSPACE_PATH}/SIH_COLAB.zip',
                                    unzip=True)

gdd.download_file_from_google_drive(file_id='1Ry5EPLyJDVIu9Qatic1tRVmxYf2EoUeg',
                                    dest_path=f'{WORKSPACE_PATH}/configs.zip',
                                    unzip=True)

!wget -qnc https://raw.githubusercontent.com/Final-Six-SIH2020/Detection-and-Classification/master/Detection/TF2/generate_tfrecords.py -P /content/workspace/

os.remove(f"{WORKSPACE_PATH}/SIH_COLAB.zip")
os.remove(f"{WORKSPACE_PATH}/configs.zip")

Unzipping...Done.
Unzipping...Done.


In [26]:
#@title generate tfrecords
! python /content/workspace/generate_tfrecords.py --csv-input={TRAIN_CSV_PATH} --output-path={TRAIN_RECORD_PATH} --image-dir={IMAGES_PATH}
! python /content/workspace/generate_tfrecords.py --csv-input={VALID_CSV_PATH} --output-path={VALID_RECORD_PATH} --image-dir={IMAGES_PATH}

2020-07-24 13:17:51.511410: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
groups: 100% 2465/2465 [00:03<00:00, 764.63it/s]
Successfully created the TFRecords: /content/workspace/annotations/train_512.record
2020-07-24 13:17:59.943022: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
groups: 100% 426/426 [00:00<00:00, 891.74it/s]
Successfully created the TFRecords: /content/workspace/annotations/val_512.record


In [27]:
#@title Generate a Label Map

labels = {
        1: 'Bus',
        2: 'Car',
        3: 'speed_sign',
        4: 'Stop_sign',
        5: 'Traffic_light',
        6: 'Traffic_sign',
        7: 'Person'
}

with open(LABEL_MAP_PATH, 'w') as f:
    # Loop through all of the labels and write each label to the file with an id
    for key, val in labels.items():
        f.write('item {\n')
        f.write(f"\tid: {key}\n")
        f.write(f"\tname: '{val}'\n")
        f.write('}\n')

with open(LABEL_MAP_PATH) as f:
    print(f.read())

item {
	id: 1
	name: 'Bus'
}
item {
	id: 2
	name: 'Car'
}
item {
	id: 3
	name: 'speed_sign'
}
item {
	id: 4
	name: 'Stop_sign'
}
item {
	id: 5
	name: 'Traffic_light'
}
item {
	id: 6
	name: 'Traffic_sign'
}
item {
	id: 7
	name: 'Person'
}



## and update Config

In [28]:
#@title Download Model and save checkpoint
import tensorflow as tf

download_base = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
model = f'{MODEL_TYPE}.tar.gz'
tmp = '/content/model.tar.gz'

!rm -rf $CHECKPOINT_DIR


if not os.path.isfile(tmp):
    tf.keras.utils.get_file(
        tmp, download_base + model
    )



!tar -xf $tmp -C $WORKSPACE_PATH
    
os.remove(tmp)
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, 'checkpoint', 'ckpt-0')


30744576/30736482 [==============================] - 0s 0us/step


In [29]:
# CONFIG

import re
from google.protobuf import text_format
from object_detection.utils import config_util
from object_detection.utils import label_map_util

CONFIG_PATH = f'/content/workspace/configs'

pipeline_skeleton = f'{CONFIG_PATH}/{CONFIG_TYPE}.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(labels.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
  'model.{}.num_classes'.format(meta_arch): num_classes,
  'train_config.batch_size': 8,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VALID_RECORD_PATH,
  'train_config.fine_tune_checkpoint': CHECKPOINT_PATH,
  'label_map_path': LABEL_MAP_PATH,
  'train_config.fine_tune_checkpoint_type': 'detection'
}

print(num_classes)
configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, WORKSPACE_PATH)

7
INFO:tensorflow:Maybe overwriting model.ssd.num_classes: 7
INFO:tensorflow:Maybe overwriting train_config.batch_size: 8
INFO:tensorflow:Maybe overwriting train_input_path: /content/workspace/annotations/train_512.record
INFO:tensorflow:Maybe overwriting eval_input_path: /content/workspace/annotations/val_512.record
INFO:tensorflow:Maybe overwriting train_config.fine_tune_checkpoint: /content/workspace/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0
INFO:tensorflow:Maybe overwriting label_map_path: /content/workspace/label_map.pbtxt
INFO:tensorflow:Maybe overwriting train_config.fine_tune_checkpoint_type: detection
INFO:tensorflow:Writing pipeline config file to /content/workspace/pipeline.config


In [30]:
# with open(f'{WORKSPACE_PATH}/pipeline.config') as f:
#     print(f.read())

## Training

In [ ]:
! rm -rf {OUTPUT_PATH}

!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=$WORKSPACE_PATH/pipeline.config \
    --model_dir={OUTPUT_PATH} \
    --alsologtostderr \
    --num_train_steps={NUM_TRAIN_STEPS} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=500

2020-07-24 13:18:13.975355: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-24 13:18:16.590158: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-24 13:18:16.614613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-24 13:18:16.615564: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-24 13:18:16.615605: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-24 13:18:16.617616: I tensorflow/stream_executor/platform/defa

## Export graph

In [ ]:
import os
import re

regex = re.compile(r"model\.ckpt-([0-9]+)\.index")
numbers = [int(regex.search(f).group(1)) for f in os.listdir(OUTPUT_PATH) if regex.search(f)]
TRAINED_CHECKPOINT_PREFIX = os.path.join(OUTPUT_PATH, 'model.ckpt-{}'.format(max(numbers)))

print(f'Using {TRAINED_CHECKPOINT_PREFIX}')

## Frozen inference graph

In [ ]:
!rm -rf $EXPORTED_PATH

!python -m object_detection.export_inference_graph \
  --pipeline_config_path=$WORKSPACE_PATH/pipeline.config \
  --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
  --output_directory=$EXPORTED_PATH

## Frozen tflite graph and conversion

In [ ]:
!rm -rf tflite_graph

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path={WORKSPACE_PATH}/pipeline.config \
    --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
    --output_directory=$WORKSPACE_PATH/tflite_graph \
    --add_postprocessing_op=true

 The outputs of the quantized model represents four arrays, named:
 * TFLite_Detetion_PostProcess    -> detection_boxes
 * TFLite_Detection_PostProcess:1 -> detection_classes
 * TFLite_Detection_PostProcess:2 -> detection_scores
 * TFLite_Detection_PostProcess:3 -> num_detections

In [ ]:
converter = tf.lite.TFLiteConverter.from_frozen_graph('/content/workspace/tflite_graph/tflite_graph.pb', 
                                                      input_shapes = {'normalized_input_image_tensor':(1,320,320, 3)},
                                                      input_arrays = ['normalized_input_image_tensor'],
                                                      output_arrays = ['TFLite_Detection_PostProcess',
                                                                       'TFLite_Detection_PostProcess:1',
                                                                       'TFLite_Detection_PostProcess:2',
                                                                       'TFLite_Detection_PostProcess:3']) 
converter.allow_custom_ops=True 
# converter.target_ops = [
#                         tf.lite.OpsSet.TFLITE_BUILTINS,
#                         tf.lite.OpsSet.SELECT_TF_OPS,

# ]
converter.optimizations = [
                        #    tf.lite.Optimize.DEFAULT,
                        #    tf.lite.Optimize.OPTIMIZE_FOR_SIZE,
                        #    tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
                           ]
tflite_model = converter.convert() 

tf_model_files = f'/content/workspace/{CONFIG_TYPE}.tflite' 
!rm -rf $tf_model_files
open(tf_model_files,"wb").write(tflite_model)

# Test the Model with TFLite Interpreter 

In [ ]:
import numpy as np

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]["shape"][1]
width = input_details[0]["shape"][2]

floating_model = input_details[0]["dtype"] == np.float32

input_mean = 127.5
input_std = 127.5

In [ ]:
print(input_details[0]["dtype"])